# Neon Films Audience Preference Analysis

## Data Collection

In [5]:
import requests
import time

# OMDB API Key
OMDB_API_KEY = "c0c5b16c"

# fetch movie data from omdb
def fetch_omdb_data(title):
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
            "(KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        )
    }
    try:
        # request data from OMDP API
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
        if data.get("Response") == "True":
            return {
                "Title": data.get("Title", ""),
                "Year": data.get("Year", ""),
                "Rated": data.get("Rated", ""),
                "Runtime": data.get("Runtime", ""),
                "imdbRating": data.get("imdbRating", ""),
                "imdbVotes": data.get("imdbVotes", ""),
                "BoxOffice": data.get("BoxOffice", ""),
                "Released": data.get("Released", ""),
                "Genre": data.get("Genre", ""),
                "Director": data.get("Director", ""),
                "Writer": data.get("Writer", ""),
                "Actors": data.get("Actors", ""),
                "Plot": data.get("Plot", ""),
                "Language": data.get("Language", ""),
                "Country": data.get("Country", ""),
                "Awards": data.get("Awards", ""),
                "Poster": data.get("Poster", ""),
                "Metascore": data.get("Metascore", ""),
                "Ratings": data.get("Ratings", "")
            }
        return {"error": data.get("Error", "Unknown error occurred")}
    # handle errors
    except requests.RequestException as e:
        return {"error": f"Request failed: {str(e)}"}

In [7]:
from trendspy import Trends
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# initialize trendsPy
tr = Trends(request_delay=10.0)

# fetch movie data from google trends
def fetch_googletrends_data(movie):
    # get movie title and release date
    title = movie["Title"].replace(":", "")
    title = movie["Released"]
    
    # convert release date to datetime format
    release_datetime = datetime.strptime(release_date, "%d %b %Y")
    # get date for one year later
    one_year_later = release_datetime + relativedelta(years=1)
    one_year_later_strftime = one_year_later.strftime("%Y-%m-%d")

    # timeframe string
    timeframe = f'{one_year_later_strftime} 1-y'
    
    try:
        # get number of movie searches
        interest_data = tr.interest_over_time(movie_title, timeframe=timeframe)
        if not interest_data.empty:
            # get total number of searches in timeframe
            total_searches = interest_data[movie_title].sum()
            return total_searches
        else:
            return -1
    except Exception as e:
        print(f"Error fetching data for '{movie_title}': {e}")
        return -1

In [ ]:
# load movie list